# Explore Movie Genre Success Patterns


This project represents the Google Data Analytics Capstone and **explores the relationship between movie genres and their different metrics in the film industry.**

![](https://nkpremices.com/content/images/size/w1100/2021/08/mih10uhu1464fx1kr0by-1.jpg)


# Introduction

*Ask phase*

The aim of this analysis is to determine which movie genres are the most common, highest-rated, attract the most attention, as well as earn the most, have the biggest budget and profit, etc. By uncovering patterns, my main objective is to gain insights into the key factors contributing to the success of various genres in the film industry. Success metrics that will be measured are: number of movies per genre, average ratings, total votes, and average budgets allocated to each genre.

**Therefore, business task of this analysis is to explore the relationship between movie genres and success metrics to uncover patterns and insights.**

**Stakeholders & Audience**

Film studios, marketing teams, and streaming platforms are interested in successful movie genres. Studios can use this data to guide their production decisions. Marketing teams tailor promotions for better campaigns, while streaming platforms improve user satisfaction by curating content based on genre preferences

# Data

*Prepare phase*

The dataset used for this analysis is [Movies Dataset](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset) made available by Rounak Banik under a [CC0 license](https://creativecommons.org/publicdomain/zero/1.0/). It has metadata about 45,000 movies, with data points including cast, crew, plot keywords, budget, revenue, ratings, release dates, languages, production companies, and more.

This dataset consists o the multiple csv files. For this purposes, I've used just the movies_metadata.csv which is the main Movies Metadata file. It contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

This dataset is an ensemble of data collected from TMDB and GroupLens. The Movie Details, Credits and Keywords have been collected from the TMDB Open API. This product uses the TMDb API but is not endorsed or certified by TMDb.

# Process

*Process phase*

The data cleaning process, documented in detail within the analysis, was conducted using R. Errors were meticulously checked, and transformations were applied to ensure data readiness for analysis. Documentation of the cleaning process ensures transparency and reproducibility for review and sharing purposes.

## Installing & Loading packages

For this project we've used following packages:

-   Tidy verse
-   GG Repel
-   Plotly

In [ ]:
library(tidyverse)
library(ggrepel)
options(scipen = 999)
library(plotly)
library(htmlwidgets)
library(repr)
library('IRdisplay')

## Importing data

As previously described, for this project purposes we've used the movies_metadata.csv.

In [ ]:
movie_dataset <- read.csv("/kaggle/input/the-movies-dataset/movies_metadata.csv")

## Data cleaning

Checking for missing values.

In [ ]:
missing_values <- sum(is.na(movie_dataset))
missing_values_by_column <- colSums(is.na(movie_dataset))

Removing 0, NA rows and duplicates

In [ ]:
movie_dataset[movie_dataset=="[]"] <- NA
movie_dataset<- na.omit(movie_dataset)
names(movie_dataset)[6]<- "m_id"
movie_dataset<-distinct(movie_dataset)

Removing not needed columns, and leaving just the ones needed for this analysis and previewing the data.

In [ ]:
movies <- subset(movie_dataset, select = -c(adult, belongs_to_collection, homepage, overview, poster_path, tagline, spoken_languages, status, video, production_countries))

Since some numeric data fields were stored as characters, we're changing it to numeric.

In [ ]:
movies$budget <- as.numeric(movies$budget)
movies$revenue <- as.numeric(movies$revenue)
movies$release_date <- as.Date(movies$release_date)

Executing various manipulations indicated by the comments. Notably, to simplify future manipulations, we have retained only the first genre for each movie, since movies in the existing data set had multiple genres separated by commas.

In [ ]:
# Removing movies with budget, revenue, vote average, and vote count zero

movies_filtered <- filter(movies, revenue > 0, budget > 0, vote_average >0, vote_count > 0)

# Leave just gender names
movies_filtered <- movies_filtered %>%
  mutate(genres = str_extract_all(genres, "\\b[A-Z]\\w*\\b") %>% lapply(toString))

# Leave just production company names
movies_filtered <- movies_filtered %>%
  mutate(production_companies = str_extract(production_companies,"\\b[A-Z]\\w*\\b") %>% lapply(toString))

# Remove Fiction since we now have Science & Fiction separately
movies_filtered <- movies_filtered %>%
  mutate(genres = str_remove(genres, "Fiction, "))

# Add Fiction to Science
movies_filtered <- movies_filtered %>%
  mutate(genres = str_replace(genres, "Science", "Science Fiction"))

# Split genres by comma, and leave just first string (genre) for easier analysis later on.

movies_filtered <-movies_filtered %>% 
  mutate(genres = str_split(genres, ", ", simplify = T)[, 1])

In the following code, doing aesthetic changes to data - reordering, removing, renaming and etc.

In [ ]:
# Remove original_title column since we don't need it. 
movies_filtered = subset(movies_filtered, select = -c(original_title) )

# Reorder columns in movies filtered list

col_order <- c("title", "genres", "production_companies",
               "original_language", "release_date", "popularity", "runtime", "budget", "revenue", "vote_average", "vote_count", "m_id", "imdb_id")
movies_filtered <- movies_filtered[, col_order]


# Rename genres to genre

movies_filtered <- movies_filtered %>% 
  rename(genre = genres)

head(movies_filtered)

Setting budget and popularity as numeric

In [ ]:
movies_filtered$budget <- as.numeric(movies_filtered$budget)
movies_filtered$popularity <- as.numeric(movies_filtered$popularity)

Since we'll need year for the analysis, I'm adding new column for year extracted from release_date

In [ ]:
movies_filtered$release_year <- year(movies_filtered$release_date)

# Analysis

*Analyze & Share phase*

To determine the most profitable genres and analyze it in the future, we need to calculate profit and ROI for each genr based on the budget and revenue already available in the data.

The formulas used for profit and ROI calculations are below:

Profit = revenue - budget

ROI = (profit / budget) \* 100

In [ ]:
# Calculate profit
movies_filtered <- movies_filtered %>%
  mutate(profit = revenue - budget)

# Calculate ROI

movies_filtered <- movies_filtered %>%
  mutate(ROI = profit / budget * 100) 


## Movies' numbers

To get the general overview about movies, we'll first get the high-level overview: of number of movies in total as well as per year.

-   Number of movies per year

-   Number of movies and average rating

In [ ]:
wp = plotly::ggplotly(movies_filtered %>%
                   ggplot(aes(x=release_year)) +
                   geom_histogram(fill="steelblue", binwidth = 1) +
                   labs(title= "Number of movies per year", x="Release year", y="Number of movies"))

saveWidget(wp, "wp.html")
display_html('<iframe src="wp.html" width=100% height=450></iframe>') #since plotly has some issues with showing in Kaggle, i've used this chunk of code to show it.

In [ ]:
movies_filtered %>%
  ggplot(aes(x=vote_average)) +
  geom_histogram(fill="steelblue", binwidth = 0.1) +
  scale_x_continuous(breaks=seq(0, 10, by=1)) +
  labs(title="Number of movies vs. average rating", x="Average vote", y="Number of movies") +
  theme(axis.text=element_text(size=14),
        axis.title=element_text(size=16,face="bold"), plot.title = element_text(size=22, face="bold"))

options(repr.plot.width = 15, repr.plot.height = 10) 


The two graphs above illustrate a significant exponential growth in the number of movie releases, particularly evident around the 2000s. Additionally, the histogram highlights that the average vote/rating for movies typically falls within the range of 6-7.

## Genre & Number of movies

Next, we'll compute the total number of movies by genre and by year.

Our first goal is to count how many movies belong to each genre.


In [ ]:
# Count number of movies and put in the separate table
genre_count <- movies_filtered %>% 
  count(genre, sort=TRUE)
  head(genre_count, 10)

In [ ]:
movies_filtered %>% group_by(genre) %>% count() %>%
  ggplot(aes(x=reorder(genre, n), y=n)) +
  geom_col(fill="steelblue") + coord_flip() +
  labs(title="Number of movies by genre", x="", y="Number of movies") +
theme(axis.text=element_text(size=14),
        axis.title=element_text(size=16,face="bold"), plot.title = element_text(size=22, face="bold"))


The most movies are belonging to drama, comedy and action as can see from the table and graph.

**Number of movies by genre by year**

Since the data in the movies_dataset is from the 1925, we will show just the data starting from 2000.

In [ ]:
# Create separate table for data from 2000-2017 and just the specific genres

  genre_yearly<- movies_filtered %>%
    filter(release_year >="2000" & release_year <="2017") %>%
    filter(genre== 'Drama' | genre == 'Comedy' | genre == 'Action' | genre == 'Adventure' | genre == 'Horror' | genre == 'Crime' | genre == 'Thriller' | genre == 'Animation' | genre == 'Fantasy' | genre == 'Romance') %>%
    group_by(genre, release_year)  %>%
    count(genre)

In [ ]:
# Plot the data
    ggplot(genre_yearly, aes(x = release_year, y = n, fill = genre)) +
      geom_col() + 
      scale_x_continuous(breaks=2000:2017) +
      labs(title="Number of movies per genre per year", x="Year", y="Number of movies") +
      theme(axis.text.x = element_text(angle=90, hjust=1,size=14), axis.text.y = element_text(size=14), 
            axis.title=element_text(size=16,face="bold"), plot.title = element_text(size=22, face="bold")) +
      guides(fill = guide_legend(reverse = TRUE))
options(repr.plot.width = 20, repr.plot.height = 10) 

As we can see from the above, the most movies per year are drama, action and comedy, and there haven't been many oscillations in their share through years.

## Genre & Rating

Since we want to know more about connection with ratings and different genres, we've created separate table genre_data where we've summarized mean popularity, mean rating and sum of votes.

**Top 5 genres**

In [ ]:
genre_data <- movies_filtered %>%
  group_by(genre) %>% 
  summarize(
    avg_vote = mean(vote_average, na.rm = TRUE),
    number_of_votes = sum(vote_count, na.rm = TRUE),
    avg_popularity = mean(popularity, na.rm = TRUE),
  )
genre_data <- genre_data %>% 
  arrange(desc(avg_vote))

# Showing top 5 genres
head(genre_data, 5)

**Bottom 5 genres**

In [ ]:
tail(genre_data)

In the tables above, we can see top and bottom movies based on avg_vote (rating).

**Western, Documentary, History, Drama and War** are the movies with the best average vote. However, what should be also taken into consideration, is that for 4 of 5 top movies, the number of **votes is quite small.**

While on the other hand, the following movies **Foreign, Horror, TV, Thriller, Action and Comedy** that the worst by average rating have rather large number of votes (Horror, Thriller, Action and Comedy).

By looking at the second table, and the observation of number of votes, we may conclude that voters would tend to respond (by voting) more on the movies they feel average, which could be interpreted that they tend to **voice opinion if the films fell short of their expectations, by giving a below-average score.**

In [ ]:
# Number of votes per genre

ggplot(genre_data, aes(x =  reorder(genre, number_of_votes), y = number_of_votes)) +
  geom_col(fill="steelblue") +
  labs(x = "Genre", y = "Number of votes", title = "Number of votes per genre") +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size=14), axis.text.y = element_text(size=14), 
        axis.title=element_text(size=16,face="bold"), plot.title = element_text(size=22, face="bold"))

Among all genres, Action, Drama, Adventure and Comedy are leading in number of votes.

In [ ]:
# Average rating per genre

ggplot(genre_data, aes(x =  reorder(genre, avg_vote), y = avg_vote)) +
  geom_col(fill="steelblue") +
  labs(x = "Genre", y = "Average vote", title = "Average vote per genre") +
   theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size=14), axis.text.y = element_text(size=14), 
        axis.title=element_text(size=16,face="bold"), plot.title = element_text(size=22, face="bold"))

As we can see, average rating per genre is similar for each genre, and is between 5 and 6.

In [ ]:
# Connection between rating & number of votes

ggplot(genre_data, aes(x=avg_vote, y=number_of_votes)) + 
  geom_point() + 
  ggrepel::geom_text_repel(aes(label = genre), size=6) +
  theme(axis.text=element_text(size=14),
        axis.title=element_text(size=16,face="bold"))

The scatterplot above confirmed the number of votes from the previous charts and tables, however it also showed connection between the rating and number of votes. If we were to look at both factors, adventure and drama have the best number of votes and avg_vote ratio.

In [ ]:
# Histogram of rating by genre

ggplot(data = movies_filtered, aes(x = vote_average)) +
  geom_histogram(bins = 30) + facet_wrap(~genre) + ggtitle("Histogram of rating by genre") + 
  xlab("Vote average") + ylab("Vote count") +
 theme(axis.text=element_text(size=14),
        axis.title=element_text(size=16,face="bold"), plot.title = element_text(size=22, face="bold"))

Popularity vs genre

In [ ]:
# Popularity vs genre 

ggplot(genre_data, aes(x =  reorder(genre, avg_popularity), y = avg_popularity)) +
  geom_col() +
  labs(x = "Genre", y = "Popularity", title = "Popularity vs Genre") +
 theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size=14), axis.text.y = element_text(size=14), 
        axis.title=element_text(size=16,face="bold"), plot.title = element_text(size=22, face="bold"))

Calculating rating per genre per year

In [ ]:
movies_last25 <- movies_filtered %>%
  filter(release_year >="2000" & release_year <="2017") %>% 
  group_by(genre, release_year)  %>%
  summarize(
    vote_average = mean(vote_average, na.rm=TRUE))


  movies_last25 %>% 
    filter(genre %in% c('Drama', 'Action', 'Comedy', 'Adventure', 'Horror', 'Crime', 'Thriller', 'Animation', 'Fantasy', 'Romance')) %>%
    group_by(genre, release_year, vote_average) %>%
    ggplot(aes(x=release_year, y=vote_average, color=genre, group=genre)) +
    geom_line() + geom_point() +
    labs(title="Average vote per genre per year", x="Year", y="Average vote") +
      theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size=14), axis.text.y = element_text(size=14), 
        axis.title=element_text(size=16,face="bold"), plot.title = element_text(size=22, face="bold"))+
    scale_x_continuous(breaks=2000:2017)

The distribution of movie genres has shown relative stability since the start of the century, with the exception of **Drama**. Drama films have experienced a slight decrease in proportion, while Thriller movies have seen a modest rise in their share.


## Genre & Financials

We're creating separate table that will contain information about average budget, revenue, profit and ROI for each genre.

In [ ]:
# Calculate average budget, revenue, profit and ROI per genre

genre_financials <- movies_filtered %>% 
  filter(budget>5000) %>%
  group_by(genre) %>% 
  summarize(
    avg_budget = mean(budget, na.rm = TRUE),
    avg_revenue = mean(revenue, na.rm = TRUE),
    avg_profit = mean(profit, na.rm = TRUE),
    avg_ROI = mean(ROI, na.rm = TRUE),
    avg_rating = mean(vote_average, na.rm = TRUE)
  )
genre_financials <- genre_financials %>% 
  arrange(desc(avg_profit))
  head(genre_financials)

In [ ]:
# Average budget per genre
ggplot(genre_financials, aes(x =  reorder(genre, avg_budget), y = avg_budget)) +
  geom_col(fill="sienna3") +
  labs(x = "Genres", y = "Average budget", title = "Average budget per genre") +
theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size=14), axis.text.y = element_text(size=14), 
        axis.title=element_text(size=16,face="bold"), plot.title = element_text(size=22, face="bold"))

In [ ]:
#Average revenue per genre
ggplot(genre_financials, aes(x =  reorder(genre, avg_revenue), y = avg_revenue)) +
  geom_col(fill="sienna3") +
  labs(x = "Genres", y = "Average revenue", title = "Average revenue vs genre") +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size=14), axis.text.y = element_text(size=14), 
        axis.title=element_text(size=16,face="bold"), plot.title = element_text(size=22, face="bold"))

In [ ]:
#Average ROI per genre
ggplot(genre_financials, aes(x =  reorder(genre, avg_ROI), y = avg_ROI)) +
  geom_col(fill="sienna3") +
  labs(x = "Genres", y = "Average ROI", title = "Average ROI vs genre") +
theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size=14), axis.text.y = element_text(size=14), 
        axis.title=element_text(size=16,face="bold"), plot.title = element_text(size=22, face="bold"))

In [ ]:
# Profit vs. genre

ggplot(genre_financials, aes(x =  reorder(genre, avg_profit), y = avg_profit)) +
  geom_col(fill="sienna3") +
  labs(x = "Genre", y = "Average profit", title = "Average profit vs genre") +
 theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size=14), axis.text.y = element_text(size=14), 
        axis.title=element_text(size=16,face="bold"), plot.title = element_text(size=22, face="bold"))

Most profitable genres and genres with the highest average revenue are **Family, Animation, Adventure, Science Fiction and Fantasy.**

Horror and Documentary genres exhibit the highest average ROI. Despite generating lower revenue and profit, Documentaries, due to their relatively low production costs, achieve a high average ROI.

### Genre Ratio

Calculating ratio - how many dollars are gotten back for each dollar invested.

In [ ]:
genre_ratio <- movies_filtered %>%
  filter(budget>5000) %>%
  group_by(genre) %>%
  summarise(profit = mean(revenue - budget),
            ratio = sum(revenue) / sum(budget),
            mean_budget = mean(budget))
genre_ratio <- genre_ratio %>% 
  arrange(desc(ratio))
  head(genre_ratio, 10)

TV is represented by only Doctor Who. If we were to ignore Doctor Who, the best movies by ratio are the ones belonging to Family, Documentary, Horror, Animation and Science Fiction.

In [ ]:
# Vote_average vs Revenue 

ggplot(genre_financials, aes(x=avg_rating, y=avg_revenue)) + 
  geom_point() + 
  ggrepel::geom_text_repel(aes(label = genre), size=6) +
  theme(axis.text=element_text(size=14),
        axis.title=element_text(size=16,face="bold"))

# Conclusion

*Act Phase*

![](https://s.yimg.com/os/creatr-uploaded-images/2020-06/f1fdd6b0-a3ee-11ea-9def-6ffa1e08d1d5)

The analysis reveals that highly-rated genres such as Western, Documentary, History, Drama, and War do not necessarily translate into the highest profits. Furthermore, there appears to be a disconnect between the average votes and total number of votes, particularly evident in genres like Action where higher total votes don't guarantee higher ratings.

On the other hand, genres like Family, Animation, Adventure, Science Fiction, and Fantasy emerge as the most profitable, boasting not only the highest average revenue but also substantial budgets. It's evident that a high average rating doesn't guarantee success in terms of revenue or profit, as seen in the case of Drama which, despite its solid ratings and large number of movies, lags behind in terms of revenue, profit, and ROI.

The analysis highlights that the safest investments in movies were found in Animation, Family, and Horror genres. Although Horror films had lower budgets compared to Animation and Family, they still generated considerable profits. Conversely, genres like History and Westerns performed notably poorer in comparison.